In [ ]:
#!pip install scikit-image

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import img_as_ubyte,measure,color,morphology, io, exposure
import os

In [ ]:
#Resize all images to 768x1024 and output to interim
for filename in os.listdir('../data/raw/example_segmentation'):
    if 'DS' in filename: 
        continue
    temp = plt.imread('../data/raw/example_segmentation/'+filename)
    temp_resized = resize(temp, (768, 1024),anti_aliasing=True)

    new_filename = filename[:5]+"re_"+filename[5:]                 
    io.imsave('../data/interim/resized_segments_ver_2/'+new_filename, img_as_ubyte(temp_resized), check_contrast = False)

In [62]:
#Create custom masks for all images (without using preexisting masks)
#If it cannot create a better mask, then default to the provided mask
images = os.listdir('../data/interim/resized_images')

#greyscaling function
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def create_masks():
    counter = 0
    for filename in images: #for images in (folder with original images)
        temp = plt.imread('../data/interim/resized_images/'+filename) #Import original image
        temp_seg = plt.imread('../data/interim/resized_segments/'+filename[:-4]+"_segmentation.png") #Import segmentation of that image

        #Slightly increase contrast for better color detection 
        temp = exposure.rescale_intensity(temp) 

        #Keep green values, blue values and greyscale images in an attempt
        #to make red colors as low as possible (since most of the lesions are red)
        gray_1 = exposure.rescale_intensity(temp[:,:,1]) #green only
        gray_2 = exposure.rescale_intensity(temp[:,:,2]) #blue only
        gray_3 = exposure.rescale_intensity(rgb2gray(temp)) #all greyscaled

        imlist = []
        difflist = []
        same_counter = 0
        struct_el = morphology.disk(2) #brush to remove most hairs
        index = 0
        for quant in np.arange(0.15,1,0.075): #Arbitrary quantiles for color histogram

            #first mask with green only
            mymask_1 = gray_1 < np.quantile(gray_1,quant) 
    
            #second mask with blue only
            mymask_2 = gray_2 < np.quantile(gray_2,quant) 
           
            #third mask with greyscaled image
            mymask_3 = gray_3 < np.quantile(gray_3,quant)
            
            #Add all masks and keep joint elements
            better_mask = mymask_1.astype('int32')+mymask_2.astype('int32')+mymask_3.astype('int32')
            fixed_mask = better_mask.copy() + temp_seg
            
            #make mask binary (or rather 0's and 1's) then make morphology corrections
            fixed_mask[fixed_mask < 4] = 0
            fixed_mask[fixed_mask == 4] = 1
            fixed_mask = fixed_mask.astype("bool")
            #Morphology transformations to generalize mask
            fixed_mask = morphology.binary_opening(fixed_mask,morphology.disk(2))
            fixed_mask = morphology.binary_closing(fixed_mask,morphology.disk(3))
            fixed_mask = morphology.binary_dilation(fixed_mask,morphology.disk(2))
            
            fixed_mask = morphology.remove_small_holes(fixed_mask,100000)
            fixed_mask = morphology.remove_small_objects(fixed_mask,3000)

            #calculate size difference between current mask and original segmentation
            diff = np.abs((np.sum(fixed_mask))/np.sum(temp_seg)) 

            #keep the fixed_mask if size is at least 5% different than the original segmentation
            #mask cannot take 100% of the image size (768x1024)
            if((diff > 1.05 or diff < 0.95) and np.sum(fixed_mask)!=786432) :
                imlist.append(fixed_mask)
                difflist.append(np.abs(1-diff))
                index+=1
                same_counter = 0
            else:
                same_counter += 1


            #break when best mask has been found
            if index >=2:
                if difflist[index-1] > difflist[index-2] or same_counter >= 3:
                    break



        #print image
        print(counter)
        counter+=1
        mask = imlist[difflist.index(min(difflist))] if imlist else temp_seg
        new_filename = filename[:-4]+"_mask.jpg"                
        io.imsave('../data/interim/custom_masks/'+new_filename, img_as_ubyte(mask), check_contrast = False)

In [59]:
%load_ext line_profiler
stats_lprun = %lprun -r -f create_masks create_masks()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [60]:
stats_lprun.print_stats();

Timer unit: 1e-07 s

Total time: 259.361 s
File: <ipython-input-57-bc08e7e23496>
Function: create_masks at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def create_masks():
    10         1         18.0     18.0      0.0      counter = 0
    11       151       3350.0     22.2      0.0      for filename in images: #for images in (folder with original images)
    12       150   14962550.0  99750.3      0.6          temp = plt.imread('../data/interim/resized_images/'+filename) #Import original image
    13       150    5369480.0  35796.5      0.2          temp_seg = plt.imread('../data/interim/resized_segments/'+filename[:-4]+"_segmentation.png") #Import segmentation of that image
    14                                           
    15                                                   #Slightly increase contrast for better color detection 
    16       150  113322802.0 755485.3      4.4          temp = exposure.res

In [61]:
create_masks()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
